## Applied Capstone Project - Week 3 Peer Graded Assignment - Toronto Neighborhood

#### Instructions and guidelines

For this assignment, we were required to explore and cluster the neighborhoods in Toronto.
There were 4 steps or questions to be tackled as follows.

1. Creating a new Notebook for this assignment, I did this and dubbed the notebook "Neighborhoods-Toronto Canada"

2. Using the Notebook to build the code to scrape a Wikipedia page in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

3. The pandas data frame should have the following:

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

4. Submit a link to your Notebook on your Github repository.

Note: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas to read the table into a pandas dataframe.

Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/

Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.

### Creating New Notebook and Importing Libraries

In [1]:
#Start by importing the libraries

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

!pip install bs4
!pip install lxml
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import json # library to handle JSON files
from pandas.io.json import json_normalize  # transform json files to pandas dataframes

!pip install folium
import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

     |████████████████████████████████| 112kB 7.7MB/s eta 0:00:01
Libraries imported.


### Scraping data from Wikipedia page and storing it to a pandas Data Frame

In [2]:
#Sending the GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#Parsing the data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

#Creating three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []

#Appending the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text.rstrip('\n'))    # .rstrip('\n') avoids new lines in postalcode cell
        boroughList.append(cells[1].text.rstrip('\n'))       # .rstrip('\n') avoids new lines in borough cell
        neighborhoodList.append(cells[2].text.rstrip('\n'))  # .rstrip('\n') avoids new lines in neighborhood cell
        
#Creating a new DataFrame from the three lists
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Drop cells having Borough value of "Not Assigned"

In [3]:
#Drop cells having borough value of "Not assigned"
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Grouping the neighborhoods in the same Borough

In [4]:
#Grouping the neighborhoods in the same borough
toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### For the neighborhoods with "Not Assigned" values, make the value the same as the Borough

In [5]:
#If Neighborhood="Not assigned", make the value the same as that of Borough
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Printing the mumber of rows

In [17]:
#Printing the number of rows of the cleaned dataframe
toronto_df_grouped.shape

(103, 3)